In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc

# --- 1. Initialization with Mobile Meta Tags ---
app = dash.Dash(
    __name__,
    external_stylesheets=[dbc.themes.BOOTSTRAP], 
    meta_tags=[
        {
            "name": "viewport",
            "content": "width=device-width, initial-scale=1, maximum-scale=1.0, user-scalable=no",
        }
    ],
    suppress_callback_exceptions=True
)

# --- 2. Responsive LIFF-Ready Layout ---
app.layout = dbc.Container(
    [
        html.H3("🏠 The Extenso Village Fund Transfer", className="text-center my-3 text-success"),
        html.Hr(),

        # --- Input Section ---
        dbc.Card(
            dbc.CardBody([
                html.P("Transfer Amount (BHT):", className="mb-1"),
                dbc.Input(id="amount-input", type="number", placeholder="e.g., 50000", className="mb-3"),

                html.P("Payment Purpose:", className="mb-1"),
                dbc.Input(id="purpose-input", type="text", placeholder="e.g., Monthly Fee", className="mb-4"),

                # --- 3. Receipt Upload Component (The Core) ---
                html.P("Upload Bank Receipt:", className="mb-1"),
                dcc.Upload(
                    id='upload-receipt',
                    children=html.Div([
                        'Drag and Drop or ',
                        html.A('Select File', className="text-info")
                    ]),
                    style={
                        'width': '100%',
                        'height': '60px',
                        'lineHeight': '60px',
                        'borderWidth': '1px',
                        'borderStyle': 'dashed',
                        'borderRadius': '5px',
                        'textAlign': 'center',
                        'marginBottom': '20px',
                        'cursor': 'pointer'
                    },
                    # We expect image files (like JPG or PNG)
                    accept='image/*', 
                    multiple=False 
                ),
                
                # Hidden element to display upload status
                html.Div(id='upload-status', className="text-muted small mb-4"),
                
                # --- 4. Submit Button ---
                dbc.Button(
                    "Submit & Close LIFF", 
                    id="submit-button", 
                    color="primary", 
                    className="w-100", # Full width button
                    n_clicks=0
                ),
                
                # Invisible component for sending commands (like closing LIFF) via JS
                html.Div(id='liff-command-output', style={'display': 'none'}),

            ])
        ),
    ],
    fluid=True # Essential for mobile/LIFF responsiveness
)

# --- Python Callbacks (Conceptual Logic) ---

# This callback handles the upload and submission logic
@app.callback(
    [dash.Output('upload-status', 'children'),
     dash.Output('liff-command-output', 'children')],
    [dash.Input('submit-button', 'n_clicks')],
    [dash.State('upload-receipt', 'contents'),
     dash.State('upload-receipt', 'filename'),
     dash.State('amount-input', 'value'),
     dash.State('purpose-input', 'value')]
)
def process_submission(n_clicks, contents, filename, amount, purpose):
    if n_clicks > 0:
        if contents is None or not amount or not purpose:
            return "❌ Please fill in all fields and upload a receipt.", None
        
        # 1. DECODE THE RECEIPT
        # In this step, you would decode the base64 string (contents) 
        # and save the file to your server or a cloud storage (e.g., Google Drive, S3).

        # 2. SAVE DATA TO DB
        # You would then save the metadata (filename, amount, purpose, and 
        # crucially, the LIFF user ID obtained via client-side JS/custom component) 
        # into your backend database.

        # 3. Trigger LIFF Close
        # Return a trigger value to the invisible component to execute JavaScript
        return "✅ Submission successful! Window is closing...", html.Script("liff.closeWindow();")
        
    return "Ready to submit.", None

if __name__ == '__main__':
    # In production for LIFF, this must be served over HTTPS
    app.run(debug=True)

C:\Users\reach\AppData\Local\Temp\ipykernel_18320\483904551.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\reach\AppData\Local\Temp\ipykernel_18320\483904551.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [ ]:
import dash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc
import pandas as pd
import base64
import io
import datetime

# --- 1. DATA SIMULATION (In-Memory Database) ---
# NOTE: House_ID uses '8801' format (no slash) for web safety.
USER_DATA = pd.DataFrame({
    'House_ID': ['8801', '8802', '8803', '8804'],
    'Password': ['1234', '5678', '0123', '1122'],
    'Full_Name': ['Pongsak', 'Arun', 'Mamee', 'Peter'],
    'Current_Balance': [-150000, 50000, 0, -300000],  # Negative means outstanding charge
    'Monthly_Charge': [50000, 50000, 75000, 100000],
    'Last_Payment': ['2025-09-01', '2025-10-01', '2025-09-15', '2025-08-20']
})

# Placeholder for uploaded receipt history
RECEIPT_HISTORY = pd.DataFrame(
    columns=['House_ID', 'Amount', 'Purpose', 'Date', 'Receipt_Filename']
)

# --- 2. DASH APPLICATION SETUP ---
app = dash.Dash(
    __name__,
    external_stylesheets=[dbc.themes.LUMEN], # A clean, light theme
    meta_tags=[
        {
            "name": "viewport",
            # CRITICAL for LIFF/mobile responsiveness
            "content": "width=device-width, initial-scale=1, maximum-scale=1.0, user-scalable=no",
        }
    ],
    suppress_callback_exceptions=True
)

# --- 3. HELPER FUNCTIONS ---

def calculate_due_date(last_payment_str):
    """Calculates the next due date 1 month after the last payment."""
    if pd.isna(last_payment_str):
        return 'N/A'
    last_payment = datetime.datetime.strptime(last_payment_str, '%Y-%m-%d').date()
    # Simple logic: due date is the 1st of the next month
    if last_payment.day > 15:
        next_month = last_payment.replace(day=1) + datetime.timedelta(days=32)
        return next_month.replace(day=1).strftime('%Y-%m-%d')
    else:
        # If payment was made early in the month, due date is the 1st of the current month (or next)
        next_month = last_payment.replace(day=1) + datetime.timedelta(days=32)
        return last_payment.replace(day=1).strftime('%Y-%m-%d')
        
def format_currency(value):
    """Formats a number as currency (using Rp for Rupiah/general currency notation)."""
    return f"Rp{abs(value):,.0f}"

def create_kpi_card(title, value, footer_text, color):
    """Creates a standardized KPI card for the dashboard."""
    return dbc.Card(
        dbc.CardBody(
            [
                html.H6(title, className="card-title text-muted"),
                html.H3(value, className=f"card-text text-{color}"),
            ],
            style={'padding': '10px'}
        ),
        dbc.CardFooter(footer_text, style={'fontSize': '0.75em'}),
        className="shadow-sm border-0 mb-3"
    )

# --- 4. LAYOUT COMPONENTS ---

login_layout = dbc.Container(
    [
        html.H4("🏡 The Extenso Village monthly maintenance fee portal", className="text-center text-primary my-4"),
        dbc.Card(
            dbc.CardBody([
                html.P("House ID:", className="mb-1"),
                dbc.Input(id="house-id-input", type="text", placeholder="e.g., 8801", className="mb-3"),
                
                html.P("Password:", className="mb-1"),
                dbc.Input(id="password-input", type="password", placeholder="Enter Password", className="mb-4"),
                
                dbc.Button("Login", id="login-button", color="success", className="w-100"),
                html.Div(id="login-status", className="mt-3 text-center")
            ])
        )
    ],
    fluid=True,
    className="p-3"
)


def create_dashboard_layout(user_data):
    """Creates the personalized dashboard and upload form."""
    user = user_data.iloc[0]
    
    # Define colors based on balance status
    balance_color = 'danger' if user['Current_Balance'] < 0 else 'success'
    balance_text = 'Outstanding' if user['Current_Balance'] < 0 else 'Credit'
    
    # Calculate next due date
    due_date = calculate_due_date(user['Last_Payment'])
    
    # Set due date color
    today = datetime.date.today()
    if due_date != 'N/A':
        due_dt = datetime.datetime.strptime(due_date, '%Y-%m-%d').date()
        if due_dt < today:
            due_color = 'danger' # Overdue
        elif due_dt <= today + datetime.timedelta(days=7):
            due_color = 'warning' # Approaching due date
        else:
            due_color = 'info' # Fine
    else:
        due_color = 'info'


    dashboard = dbc.Container(
        [
            html.H5(f"Welcome, {user['Full_Name']}!", className="text-center my-3 text-info"),
            html.Hr(className="mb-4"),

            # --- Dashboard KPIs (Responsive Grid) ---
            dbc.Row(
                [
                    dbc.Col(
                        create_kpi_card(
                            "Current Balance",
                            format_currency(user['Current_Balance']),
                            f"Status: {balance_text}",
                            balance_color
                        ),
                        xs=12, md=4
                    ),
                    dbc.Col(
                        create_kpi_card(
                            "Monthly Charge",
                            format_currency(user['Monthly_Charge']),
                            "Standard contribution amount.",
                            'primary'
                        ),
                        xs=12, md=4
                    ),
                    dbc.Col(
                        create_kpi_card(
                            "Next Payment Due",
                            due_date,
                            "Based on last recorded payment.",
                            due_color
                        ),
                        xs=12, md=4
                    ),
                ]
            ),
            
            html.H5("💸 Submit New Receipt", className="mt-4 mb-3 text-primary"),

            # --- Receipt Upload Form ---
            dbc.Card(
                dbc.CardBody([
                    html.P("Transfer Amount:", className="mb-1"),
                    dbc.Input(id="amount-input", type="number", placeholder="e.g., 50000", className="mb-3"),

                    html.P("Payment Purpose:", className="mb-1"),
                    dbc.Input(id="purpose-input", type="text", placeholder="e.g., Monthly Fee", className="mb-4"),

                    # File Upload Component
                    dcc.Upload(
                        id='upload-receipt',
                        children=html.Div([
                            'Drag and Drop or ',
                            html.A('Select Receipt File', className="text-info")
                        ]),
                        style={
                            'width': '100%', 'height': '60px', 'lineHeight': '60px',
                            'borderWidth': '1px', 'borderStyle': 'dashed', 'borderRadius': '5px',
                            'textAlign': 'center', 'marginBottom': '20px', 'cursor': 'pointer'
                        },
                        accept='image/*', 
                        multiple=False 
                    ),
                    
                    html.Div(id='upload-status', className="text-muted small mb-4"),
                    
                    dbc.Button(
                        "Submit & Close LIFF", 
                        id="submit-button", 
                        color="success", 
                        className="w-100", 
                        n_clicks=0
                    ),
                ])
            ),
            
            # --- Hidden components for data and LIFF control ---
            # dcc.Store holds the logged-in user's ID
            dcc.Store(id='current-house-id', data=user['House_ID']),
            # This hidden div is used to trigger the JS LIFF close command
            html.Div(id='liff-command-output', style={'display': 'none'}),
        ],
        fluid=True,
        className="p-3"
    )
    return dashboard

# --- 5. APP LAYOUT AND ROUTING ---
app.layout = html.Div(
    [
        dcc.Location(id='url', refresh=False),
        html.Div(id='page-content', children=login_layout)
    ]
)

# --- 6. CALLBACKS ---

# Callback for Login (FIXED to handle initial load and whitespace)
@app.callback(
    [Output('page-content', 'children'),
     Output('login-status', 'children')],
    [Input('login-button', 'n_clicks')],
    [State('house-id-input', 'value'),
     State('password-input', 'value')]
)
def authenticate_user(n_clicks, house_id, password):
    # Handle initial load: use dash.no_update to prevent the callback from trying to process None
    if n_clicks is None or n_clicks == 0:
        return dash.no_update, ""
        
    # Robust Input Handling: Strip whitespace and safely handle None
    input_house_id = str(house_id).strip().lower() if house_id else ''
    input_password = str(password).strip() if password else ''

    # Check for empty submission after cleaning
    if not input_house_id or not input_password:
        return dash.no_update, dbc.Alert("Please enter both ID and Password.", color="warning")

    # Authentication Logic
    user_match = USER_DATA[
        (USER_DATA['House_ID'].str.lower() == input_house_id) & 
        (USER_DATA['Password'] == input_password)
    ]
    
    if not user_match.empty:
        # Login successful
        return create_dashboard_layout(user_match), ""
    else:
        # Login failed
        return login_layout, dbc.Alert("Invalid House ID or Password.", color="danger")


# Callback for Receipt Submission
@app.callback(
    [Output('upload-status', 'children'),
     Output('liff-command-output', 'children'),
     Output('page-content', 'children', allow_duplicate=True)],
    [Input('submit-button', 'n_clicks')],
    [State('upload-receipt', 'contents'),
     State('upload-receipt', 'filename'),
     State('amount-input', 'value'),
     State('purpose-input', 'value'),
     State('current-house-id', 'data')],
    prevent_initial_call=True
)
def process_submission(n_clicks, contents, filename, amount, purpose, house_id):
    global USER_DATA, RECEIPT_HISTORY
    
    if n_clicks > 0:
        if contents is None or not amount or not purpose:
            return "❌ Please fill in all fields and upload a receipt.", None, dash.no_update
        
        # --- 1. PROCESS RECEIPT (File Handling Simulation) ---
        # The 'contents' is the base64 string. In a live system, this is where you:
        # 1. Decode base64 to bytes.
        # 2. Upload the bytes to a cloud storage (S3, GCS, etc.) using the House_ID in the path.
        # 3. Save the *URL* to the cloud file in your database.
        
        # This is a placeholder for the uploaded file:
        # 
        
        # --- 2. UPDATE FINANCIAL DATA ---
        try:
            amount = int(amount)
        except ValueError:
            return "❌ Amount must be a valid number.", None, dash.no_update

        # Update the Current_Balance
        USER_DATA.loc[USER_DATA['House_ID'] == house_id, 'Current_Balance'] += amount
        USER_DATA.loc[USER_DATA['House_ID'] == house_id, 'Last_Payment'] = datetime.date.today().strftime('%Y-%m-%d')

        # Add transaction to history
        new_receipt = pd.DataFrame([{
            'House_ID': house_id,
            'Amount': amount,
            'Purpose': purpose,
            'Date': datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'Receipt_Filename': filename
        }])
        RECEIPT_HISTORY = pd.concat([RECEIPT_HISTORY, new_receipt], ignore_index=True)

        # --- 3. TRIGGER LIFF CLOSE ---
        # This JavaScript snippet will close the LIFF window immediately after submission.
        liff_script = html.Script("if (liff && liff.isInClient()) { liff.closeWindow(); }")
        
        return (
            "✅ Submission successful! Window is closing...", 
            liff_script, 
            dash.no_update # We just close, no need to redraw dashboard
        )

    return dash.no_update, dash.no_update, dash.no_update


if __name__ == '__main__':
    # REMINDER: For real LIFF integration, you MUST serve this app over HTTPS.
    # Use ngrok for testing: ngrok http 8055
    app.run_server(debug=True, port=8055, output = 'external')

[2025-10-11 17:56:17,373] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\Users\reach\miniforge3\Lib\site-packages\flask\app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\reach\miniforge3\Lib\site-packages\flask\app.py", line 865, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\reach\miniforge3\Lib\site-packages\dash\dash.py", line 1352, in dispatch
    ctx.run(
  File "c:\Users\reach\miniforge3\Lib\site-packages\dash\_callback.py", line 450, in add_context
    output_value = _invoke_callback(func, *func_args, **func_kwargs)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\reach\miniforge3\Lib\site-packages\dash\_callback.py", line 39, in _invoke_c